In [1]:
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain.vectorstores import Qdrant
import os
import pinecone
from tqdm.autonotebook import tqdm

c:\Users\Kwadwo Silas\Desktop\miscellenous\chatbot\env\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
os.environ["COHERE_API_KEY"] = "P5qlLVKqvPGIixGiGVmrKe1yXEQIbrcFoNMJn5ax"
# initailize pinecode 

pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),  # find at app.pinecone.io
    environment=os.getenv("PINECONE_ENV"),  # next to api key in console
)

In [43]:
# load our document here 
from langchain.document_loaders import TextLoader

loader = TextLoader(file_path="./gtbank_data_web.txt", encoding='utf8')

data = loader.load()

In [5]:
# load our document here 
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="./gtbank-data-web.csv", csv_args={'delimiter': ','})

data = loader.load()

In [44]:
# split texts into chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1200,
    chunk_overlap  = 200,
    length_function = len,
)

data_chunks = text_splitter.split_documents(data)

In [45]:
data_chunks[0:1]

[Document(page_content='Innovative Banking Products\nTailored to your lifestyle, designed for your\npersonal and business needs.\nPersonal Banking products\nEasy Savers Account\nThe Easy Savers Account is a crossbreed of a current and savings account with added benefits just for you. Enjoy the flexibility of a current account and the interest-bearing feature of a savings account.\nEarn interest\nFree e-Banking services\nPay in cheques and dividends\nOptional COT\nFeatures & Benefits\nEarn a competitive interest pegged at prevailing savings account rate.\nReliable electronic banking services to enable you transact from your home or office (minimal charge of GHS2 monthly)\nOptional COT (charged if withdrawals exceed four times in a month)\nPersonalized cheque book.\nFlexibility of paying into own account via your Mobile Money wallet.\nTerms and Conditions\nNo minimum opening balance (Zero balance account)\nNo minimum account balance\nThe Easy Savers account is a non-borrowing account.\nH

In [46]:
len(data_chunks)

120

In [47]:
# initialize embeddings 
embeddings = CohereEmbeddings(model = "multilingual-22-12")

In [48]:
# vector storage

db = Qdrant.from_documents(data_chunks, embeddings, location=":memory:", collection_name="my_documents", distance_func="Cosine")

In [49]:
db.similarity_search("cards")

[Document(page_content='GTBank Card Terms and Conditions\nGTBank Card Terms and Conditions\n\nI/We hereby apply for a GTBank Card (the “Card”). I/We declare that the information provided\n\nin this application is true and correct and shall advise you of any changes thereto. I/We hereby authorize GTBank to verify any information from whatever sources it may consider appropriate.\n\nI/We also agree to be bound by the following terms and conditions:\n\n1. I/We undertake that this application, signed by me/us, is for the issuance of a Guaranty Trust Bank\n\n(Ghana) Ltd (GTBank) Card for myself/ourselves and for my/our use and that in doing so, I/we do not represent the interest of someone else.\n\n2. I/We understand that GTBank may decline my/our application without any reasons given us.\n\n3. I/We understand that the Card remains the property of GTBank and I/we shall surrender it\n\nUnconditionally and without reservation upon demand by GTBank.\n\n4. I/We undertake not to use or attempt t

In [17]:
# make our prompt 
prompt_template = """

generate response to the question based on the text provided.

Change instances where the bank is HDFC Bank to GTbank, 

If the text doesn't contain the answer, reply that the answer is not available and can request for more assistance by contacting us by telephone or sending a mail to customer service representative.

the Telephone Numbers:Tel: (+233) 302 611 560 Toll free: 0800 124 000 and the mail is gh.customersupport@gtbank.com

Text: {context}

Question: {question}
"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

In [53]:
question = "Tell me about all the visa cards available at gtbank"

In [54]:
# question and answer here 
qa = RetrievalQA.from_chain_type(llm=Cohere(model="command-nightly", temperature=0), 
                                 chain_type="stuff", 
                                 retriever=db.as_retriever(search_type='mmr'),
                                 chain_type_kwargs=chain_type_kwargs, 
                                 return_source_documents=True)
                                 

answer = qa({"query": question})

In [55]:
print(answer['result'])

 GTBank offers a range of visa cards, including the GTBank Classic Visa Card, the GTBank Gold Visa Card, and the GTBank Platinum Visa Card. The Classic Visa Card is designed for everyday use, with benefits such as purchase protection, extended warranty, and emergency card replacement. The Gold Visa Card is a premium card that offers more exclusive benefits, such as airport lounge access and a higher transaction limit. The Platinum Visa Card is the most exclusive card offered by GTBank, with benefits such as a personal assistant and a 24/7 concierge service. All GTBank Visa cards offer the convenience of being able to make purchases and withdrawals both in Ghana and abroad. 

Please note that the answers provided by me are based on the information available at the time of response and may not always be up-to-date. If you require more accurate and real-time information, it would be best to contact GTBank directly by telephone at +233 302 611 560 or toll-free at 0800 124 000, or by email 